In [1]:
import numpy as np
import pandas as pd

In [3]:
df1=pd.read_csv("../r2Data/user_vendor_series_data.csv",usecols=['user_id','business_id','stars_x','text','city','latitude','longitude','review_count','categories'],nrows=100000,keep_default_na=False)

## Buliding basic collaborative model

In [15]:
df1.drop(columns=['address','useful','funny','cool'],inplace=True)

In [33]:
df2=df1.iloc[:20000,:]

In [5]:
df1.head(3)

,user_id,business_id,stars_x,text,city,latitude,longitude,review_count,categories
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",North Wales,40.210196,-75.223639,169,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,Philadelphia,39.952103,-75.172753,144,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,Tucson,32.207233,-110.980864,47,"Restaurants, Breakfast & Brunch"


In [7]:
pivot_table_df=pd.pivot_table(df1,index=['business_id'],columns=['user_id'],values='stars_x')

In [8]:
pivot_table_df.fillna(value=0,inplace=True)

In [9]:
pivot_table_df

user_id,---2PmXbF47D870stH1jqA,--4AjktZiHowEIBCMd4CZA,--E0uVPphTORm_OiZ5KCvA,--KMTwCrhKKUmr7riuS4WQ,--S8M395r8NtOCvS2LRfDw,--UizzbnQlZg7bEv2oXEyg,--WhhlIim6ISY5yhFYZreg,--_H9j6ggxvqhh9nPofZwg,--_r6E98SNIrGU7weyNxbw,--ccVMj2PN6Z9qtdOdlung,...,zzeRzizkihWHz9bVAvbcVw,zzgLgszu48af4NJULymzAA,zzgMuJgxmToqcJ5iu1TngQ,zziJLt25YU6dp01sewR-IQ,zzjThJ4A1m1N78gquSR_QA,zzljYL9c22HWRO3VvJ1_Pg,zzsPf8xNW11nd0B6MZqfRw,zzsqjDvanJhH9tn8NautOQ,zztkCqqgR6VntYbqio4UTQ,zzxUT-fFNXpp5i5xJHmvpA
business_id,,,,,,,,,,,,,,,,,,,,,
--ZVrH2X2QXBFdCilbirsw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--_9CAxgfXZmoFdNIRrhHA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-02xFuruu85XmDn2xiynJw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0Ym1Wg3bXd_TDz8JtvOQg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0fvhILrC9UsQ6gLNpZlTQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyKTy79BwgCLdUGBiuYPxA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzIF9qp2UoHN48EeZH_IDg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzKVqizyl3QCb3GZmg_cNg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from scipy.sparse import csr_matrix

In [11]:
df_sparse=csr_matrix(pivot_table_df)

In [12]:
from sklearn.neighbors import NearestNeighbors
model1=NearestNeighbors(algorithm='brute')

In [13]:
model1.fit(df_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
distance,rec=model1.kneighbors(pivot_table_df.iloc[])